In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/CW2Code/

/content/drive/MyDrive/CW2Code


In [ ]:
# !unzip -q "glove_twitter.zip"

replace glove.twitter.27B.25d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace glove.twitter.27B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace glove.twitter.27B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
# !unzip -q "Sentiment140.zip"

# Import Libraries

In [ ]:
!pip install contractions
!pip install emoji --upgrade
!pip install emot
!pip install tensorflow keras

  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234925 sha256=17570ee5b0c42e597f48fad90fbbd0f9470c5db9da07faaf61c960b2115451a6
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\9a\b8\0f\f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
scipy 1.7.1 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.2 which is incompatible.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.24.2 which is incompatible.


In [ ]:
import contractions
import pandas as pd
import emoji
import re
import emot
import string
import codecs
import numpy as np
import html
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

import pickle

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, Dropout, Flatten
from tensorflow.keras.models import load_model

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data Loading

In [ ]:
columns  = ["sentiment", "ids", "date", "flag", "user", "text"]

In [ ]:
dataset = pd.read_csv('./training.1600000.processed.noemoticon.csv', encoding="ISO-8859-1" , names=columns)

In [ ]:
# Drop irrelavant columns from dataset dataframe
dataset = dataset.drop(['ids', 'date','flag','user'], axis=1)

In [ ]:
dataset['sentiment'] = dataset['sentiment'].replace(4,1)

In [ ]:
dataset

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


# Data PreProcessing

In [ ]:
def remove_urls(tweet):
    # Removal of urls since may not be relevant to the content for analysis
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    return url_pattern.sub(r'', tweet)

In [ ]:
def clean_mentions(tweet):
    # Remove twitter mentions i.e. username since may not be relevant to the content and
    # thus extraneous information
    return re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])",'',tweet)

In [ ]:
def expand_contractions(tweet):
    # Expand shortened version of phrases or words
    tweet = contractions.fix(tweet)

    return tweet

In [ ]:
def replace_numbers(tweet):
    # Removal of numbers from tweet
    return re.sub('[0-9]+', '', tweet)

In [ ]:
slang_dict = {
    'af': 'as fuck',
    'afaik': 'as far as I know',
    'amirite': 'am I right',
    'b4': 'before',
    'bc': 'because',
    'b/c': 'because',
    'btw': 'by the way',
    'brb': 'be right back',
    'cya': 'see ya',
    'da': 'the',
    'deets': 'details',
    'dope': 'cool',
    'fam': 'family',
    'fb': 'Facebook',
    'fml': 'fuck my life',
    'ftw': 'for the win',
    'gf': 'girlfriend',
    'gj': 'good job',
    'gr8': 'great',
    'h8': 'hate',
    'hmu': 'hit me up',
    'idc': 'I don’t care',
    'idk': 'I don’t know',
    'ikr': 'I know right',
    'imho': 'in my humble opinion',
    'imo': 'in my opinion',
    'irl': 'in real life',
    'j/k': 'just kidding',
    'jk': 'just kidding',
    'k': 'okay',
    'lmao': 'laughing my ass off',
    'lmk': 'let me know',
    'lol': 'laugh out loud',
    'm8': 'mate',
    'nbd': 'no big deal',
    'ngl': 'not gonna lie',
    'nm': 'nevermind',
    'np': 'no problem',
    'nsfw': 'not safe for work',
    'omg': 'oh my god',
    'ppl': 'people',
    'probs': 'probably',
    'qt': 'cutie',
    'rly': 'really',
    'rofl': 'rolling on the floor laughing',
    'smh': 'shaking my head',
    'sry': 'sorry',
    'stfu': 'shut the fuck up',
    'tbh': 'to be honest',
    'tho': 'though',
    'thx': 'thanks',
    'tmi': 'too much information',
    'ttyl': 'talk to you later',
    'u': 'you',
    'ur': 'your',
    'w/': 'with',
    'w/o': 'without',
    'wbu': 'what about you',
    'wtf': 'what the fuck',
    'wyd': 'what are you doing',
    'y': 'why',
    'yolo': 'you only live once',
    'yr': 'year'
}

In [ ]:
def replace_slangs_words(tweet, slang_dict):
    # Replace slag words as abbreviation to its corresponding english phrase
    tweet_wo_slangs = []
    for word in tweet.split():
        if word in slang_dict:
            tweet_wo_slangs.append(slang_dict[word])
        else:
            tweet_wo_slangs.append(word)

    return ' '.join(tweet_wo_slangs)

In [ ]:
nonstandard_dict = {
    'aint': 'am not',
    'gonna': 'going to',
    'gotta': 'got to',
    'hafta': 'have to',
    'wanna': 'want to',
    'coulda': 'could have',
    'woulda': 'would have',
    'shoulda': 'should have',
    'mighta': 'might have',
    'musta': 'must have',
    'ain\'t': 'am not',
    'kinda': 'kind of',
    'sorta': 'sort of'
}

In [ ]:
def replace_nonstandard(tweet, nonstandard_dict):
    # To replace slag words which are non-english words to their corresponding english words
    tweet_wo_nonstandard = []
    for word in tweet.split():
        if word in nonstandard_dict:
            tweet_wo_nonstandard.append(nonstandard_dict[word])
        else:
            tweet_wo_nonstandard.append(word)

    return ' '.join(tweet_wo_nonstandard)

In [ ]:
def convert_emojis(tweet):
  # convert emoji to words to capture intended meaning of emoji
  tweet = emoji.demojize(tweet)
  tweet = re.sub('_',' ',tweet)

  return tweet

In [ ]:
def convert_emoticons(tweet):
    # convert emoticons to words to capture intended meaning of emoticons
    emo_obj = emot.EMOTICONS_EMO
    for emo in emo_obj:
        escaped_emot = re.escape(emo)
        tweet = re.sub(u'({})'.format(escaped_emot), " ".join(emo_obj[emo].replace(",","").split()), tweet)

    return tweet

In [ ]:
punctuations = string.punctuation
def remove_punctuations(tweet):
    # Removal of punctuations from tweets
    return tweet.translate(str.maketrans('', '', punctuations))

In [ ]:
def load_file(path):
    file_text = ''
    file = codecs.open(path, 'r', encoding = 'utf-8')
    file_lines = file.readlines()
    for line in file_lines:
        # Text cleaning, remove any whitespace lines
        line = line.replace('\n','')
        file_text = file_text + line
    file.close()

    return file_text

In [ ]:
# loads contents of a stopwords file
StopWords = load_file('./Stopwords.txt')
StopWords_list = re.split(r"[~\r\n]+", StopWords)

In [ ]:
def remove_stopwords(tweet):
    # Removal of stopwords from tweets which have no or less meaning to reduce noise
    return " ".join([word for word in str(tweet).split() if word not in StopWords_list])

In [ ]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(tweet):
  # Tag each word in the tweet with its corresponding part-of-speech.
  pos_tagged_tweet = nltk.pos_tag(tweet.split())
  # lemmatizer is applied to each word in the tweet using the WordNet POS tag. If the POS is not found in the wordnet dict, the default tag used is NOUN.
  return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_tweet])

In [ ]:
def split_hash(tweet):
  # Find all hashtags in the tweet
  hashtags = re.findall(r'#\w+', tweet)
  # Loop through each hashtag and split it into individual words
  for hashtag in hashtags:
      # Remove the "#" character from the hashtag
      hashtag_text = hashtag[1:]
      # Use regular expressions to split the hashtag into individual words based on CamelCase eg: 'ElectricVehcile' into 'Electric Vehicle'
      words = re.findall(r'[A-Z]?[a-z]+', hashtag_text)
      # Combine the words back into a single string and replace the original hashtag in the tweet with the new, split hashtag
      split_hashtag = ' '.join(words).lower()
      tweet = tweet.replace(hashtag, split_hashtag)

  return tweet

In [ ]:
def remove_escape_sequence(tweet):
    # Remove escape sequences from tweet to avoid issues such avoid addition of extra spaces
    tweet = tweet.replace('\n', '')
    tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)
    return tweet

In [ ]:
def preprocessing_tweets(tweet):
    # Preprocessing steps for sentiment analysis
    tweet = split_hash(tweet)
    tweet = remove_urls(tweet)
    tweet = remove_escape_sequence(tweet)
    tweet = html.unescape(tweet)
    tweet = expand_contractions(tweet)
    tweet = convert_emoticons(tweet)
    tweet = convert_emojis(tweet)
    tweet = tweet.replace('::',' ')
    tweet = replace_numbers(tweet)
    tweet = clean_mentions(tweet)
    tweet = replace_slangs_words(tweet, slang_dict)
    tweet = replace_nonstandard(tweet, nonstandard_dict)
    tweet = remove_punctuations(tweet)
    tweet = lemmatize_words(tweet)
    tweet = tweet.lower()
    tweet = remove_stopwords(tweet)
    return (tweet)

In [ ]:
dataset['text'] = dataset['text'].apply(lambda x:preprocessing_tweets(x))

In [ ]:
dataset = dataset.dropna() # There may be tweets with only mentions therefore after preprocessing results in nan

In [ ]:
dataset

,sentiment,text
0,0,awww bummer get david carr third day wink smirk
1,0,upset cannot update facebook texting might cry...
2,0,dive many time ball managed save rest go bound
3,0,whole body feel itchy like fire
4,0,behave mad cannot see
...,...,...
1599995,1,wake school best feeling ever
1599996,1,thewdbcom cool hear old walt interview
1599997,1,ready mojo makeover ask detail
1599998,1,happy th birthday boo alll time tupac amaru sh...


In [ ]:
dataset.to_csv('Preprocessed16.csv', index = False) # To be removed

In [ ]:
# from collections import Counter
# cnt = Counter()
# for text in dataset["text"].values:
#     for word in text.split():
#         cnt[word] += 1

# cnt.most_common(10)

In [ ]:
# sen ="&quot; Hi My name is 🤣😂😀❤😍😁👍😒👌🤷‍♂😊😘 :/ :)\t😄 ____ \y\n wouldn' t https://www.google.com \t\n saying\t way can't #ElectricVehicle #EV #EEE @nfdwe \n@kwenrf234 123 yolo can't wanna lawda harsh.&quot;"
# print(preprocessing_tweets(sen))

# Feature Extraction

In [ ]:
dataset = pd.read_csv('./Preprocessed16.csv') #TO BE REMOVED

In [ ]:
token = Tokenizer()

In [ ]:
# Tokenize tweets and add unique words to the vocabulary
def getVocab(tweets):
  vocab = set()
  for tweet in tweets:
      words = word_tokenize(tweet)
      vocab.update(words)
  return vocab

In [ ]:
vocab = getVocab(dataset['text'])

In [ ]:
# Sort vocab of tweet dataset
vocab = sorted(vocab)

In [ ]:
len(vocab) #TO BE REMOVED

391031

In [ ]:
# Prune pre-trained embeddings
def optimise_glove(gloveText,vocab):
  # Takes dictionary of glove word vectors as input and returns a new dictionary containing only those words
  # that are present in a given set of unique words.
  newGlove= dict()
  for word, vector in gloveText.items():
    if word in vocab or word =='unk':
      newGlove[word] = gloveText[word]
  return newGlove

In [ ]:
def create_vocab_to_dict(words):
    # Convert vocabulary of tweet into dictionary having keys as words and values as index of that word
    vocab_to_dict = {}
    index = 0
    for word in words:
        if word not in vocab_to_dict:
            vocab_to_dict[word] = index
            index += 1
    return vocab_to_dict

In [ ]:
vocab_dict = create_vocab_to_dict(vocab)

Use of a pre-trained Twitter GloVe model with embedding space of 100 from Stanford for Sentiment Analysis task on Twitter data.
> Pre-trained GloVe Model: Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 25d, 50d, 100d, & 200d vectors, 1.42 GB download)

> Link: https://nlp.stanford.edu/projects/glove/

> Note:For the glove file Download the file from: https://drive.google.com/file/d/1-6EjyzaKqXjlMZduUiC8-W9K4VI8PXxx/view?usp=share_link


In [ ]:
def creatingEmbeddingMatrix(vocab_dict):
  # To load pretrained glove word vectors file
  with open('./glove.twitter.27B.100d.txt','r',encoding='UTF-8') as f:
    glove_model = {}
    for line in f:
      split_line = line.split()
      word = split_line[0]
      embedding = np.array(split_line[1:], dtype=np.float32)
      glove_model[word] = embedding
    print(len(glove_model))
      # Removing words which are not present in the gloveText file
    opt_glove_model = optimise_glove(glove_model,vocab_dict)
      # Getting list of vocab(unique words) which are present in gloveText file
      # Creating matrix of 0s of (m,n) where,
      # m is the number of unique words which are present in gloveText file + 1 (Adding padding vector at start)
      # n is the embedding space which is of length 300.
    embedding_matrix = np.zeros((len(opt_glove_model)+1,100))
    i = 1
      # Creation of embedding matrix(vectors) of all the matching vocab words for embedding layer
    for word in list(opt_glove_model.keys()):
      embedding_matrix[i] = np.asarray(opt_glove_model[word], 'float32')
      i+=1
    embedding_matrix = np.asarray(embedding_matrix, 'float32')
    return embedding_matrix, opt_glove_model

In [ ]:
embedding_matrix, pruned_glove_model = creatingEmbeddingMatrix(vocab_dict)

1193514


In [ ]:
print(len(pruned_glove_model)) #TO BE REMOVED

124539


In [ ]:
# Save new pruned model to a file
with open("pruned_glove_model.txt", "w", encoding="utf-8") as f:
    for word, vector in pruned_glove_model.items():
        f.write(f"{word} {' '.join(str(x) for x in vector)}\n")

In [ ]:
pruned_glove_model_dict = create_vocab_to_dict(list(pruned_glove_model))

In [ ]:
len(pruned_glove_model_dict) #TO BE REMOVED

124539

In [ ]:
max_tweet_len = 60
unk_index =  list(pruned_glove_model_dict).index('unk')

In [ ]:
unk_index #TO BE REMOVED

29393

In [ ]:
pruned_glove_model_dict['unk'] #TO BE REMOVED

29393

In [ ]:
with open('Pruned_gloveModel.pkl', 'wb') as f:  # Open a text file
    pickle.dump(pruned_glove_model_dict, f) # Serialize the list
    f.close()

In [ ]:
def getInput(tweets, words, max_tweet_len, unk_index):
    X = []
    for tweet in tqdm(tweets):
        idx = [words[word]+1  if word in words else unk_index+1 for word in tweet.split()]
        X.append(idx)
    X = pad_sequences(X, maxlen=max_tweet_len, padding='post', truncating='post', value=0)
    return X

In [ ]:
X = getInput(dataset['text'], pruned_glove_model_dict, max_tweet_len, unk_index)

100%|████████████████████████████████████████████████████████████████████| 1592240/1592240 [00:08<00:00, 183998.19it/s]


In [ ]:
dataset['text'][500:510] #TO BE REMOVED

500    list item ebay sell takes forevermeanwhile col...
501            get presentation slide cry week hard week
502                                                 lose
503    like previously skin thing start season two li...
504             wish therei pretty good scar shit people
505                   going sound vain run fav lip gloss
506    never click link scream spider spider know wel...
507                                   time work get sick
508                           imac come indonesia states
509                    dammit need stop buying furniture
Name: text, dtype: object

In [ ]:
embedding_matrix[1435] #TO BE REMOVED

array([-7.5353e-02,  3.6746e-01,  1.5601e-01, -3.0391e-01,  6.6648e-01,
       -2.7182e-01,  7.0206e-01, -3.3533e-01, -1.4386e-01,  2.7116e-02,
        2.1675e-02, -1.0689e-01, -3.3964e+00,  2.7311e-01, -7.3037e-02,
        3.0913e-01,  4.0654e-01, -2.1790e-03,  7.8056e-03,  6.4235e-02,
       -2.9094e-01, -4.0283e-02, -3.1580e-01,  2.3444e-01, -5.6632e-01,
       -1.4728e-01,  3.6431e-01, -8.0864e-01,  9.0580e-01,  8.2974e-02,
       -8.3946e-02,  1.5435e-01, -4.3824e-01, -1.0304e-01,  7.6613e-01,
       -9.4312e-01,  1.0410e+00,  1.4602e-01, -1.6489e-01, -1.9296e-01,
       -3.1341e-02,  1.5007e-02,  1.7113e-01, -3.3630e-01,  1.3170e-01,
       -5.7878e-01, -6.8777e-02, -7.7445e-01, -3.2352e-01,  3.5709e-01,
       -1.0262e-02, -1.7327e-01, -5.7990e-01,  2.0623e-01, -2.0086e-01,
        8.2070e-01, -5.3029e-03, -1.6298e-01, -2.4454e-01, -2.0762e-01,
       -3.3225e-01, -7.6951e-02,  5.5174e-01,  2.5978e-01, -1.6672e-01,
        3.9690e-01, -2.3708e-01, -6.9310e-01, -8.9850e-02, -1.19

In [ ]:
pruned_glove_model['list'] #TO BE REMOVED

array([-7.5353e-02,  3.6746e-01,  1.5601e-01, -3.0391e-01,  6.6648e-01,
       -2.7182e-01,  7.0206e-01, -3.3533e-01, -1.4386e-01,  2.7116e-02,
        2.1675e-02, -1.0689e-01, -3.3964e+00,  2.7311e-01, -7.3037e-02,
        3.0913e-01,  4.0654e-01, -2.1790e-03,  7.8056e-03,  6.4235e-02,
       -2.9094e-01, -4.0283e-02, -3.1580e-01,  2.3444e-01, -5.6632e-01,
       -1.4728e-01,  3.6431e-01, -8.0864e-01,  9.0580e-01,  8.2974e-02,
       -8.3946e-02,  1.5435e-01, -4.3824e-01, -1.0304e-01,  7.6613e-01,
       -9.4312e-01,  1.0410e+00,  1.4602e-01, -1.6489e-01, -1.9296e-01,
       -3.1341e-02,  1.5007e-02,  1.7113e-01, -3.3630e-01,  1.3170e-01,
       -5.7878e-01, -6.8777e-02, -7.7445e-01, -3.2352e-01,  3.5709e-01,
       -1.0262e-02, -1.7327e-01, -5.7990e-01,  2.0623e-01, -2.0086e-01,
        8.2070e-01, -5.3029e-03, -1.6298e-01, -2.4454e-01, -2.0762e-01,
       -3.3225e-01, -7.6951e-02,  5.5174e-01,  2.5978e-01, -1.6672e-01,
        3.9690e-01, -2.3708e-01, -6.9310e-01, -8.9850e-02, -1.19

In [ ]:
list(pruned_glove_model)[1434] #TO BE REMOVED

'list'

In [ ]:
X[500:520] #TO BE REMOVED

array([[  1435,   5056,   6450, ...,      0,      0,      0],
       [    24,   8490,   5855, ...,      0,      0,      0],
       [   806,      0,      0, ...,      0,      0,      0],
       ...,
       [101154,   1155,   1587, ...,      0,      0,      0],
       [ 26174,     64,   1440, ...,      0,      0,      0],
       [   406,    418,    889, ...,      0,      0,      0]])

In [ ]:
def getOutput(sentiments):
    sentiment_array = []
    for sentiment in sentiments:
        sentiment_array.append(sentiment)
    return np.array(sentiment_array)

In [ ]:
Y = getOutput(dataset['sentiment'])

# Data Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.20,shuffle=True)

# BiLSTM Model

In [ ]:
# Define the BiLSTM model
model = Sequential()
# Embedding Layer
model.add(Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1], input_length=X.shape[1], trainable=False))
# BiLSTM Layer
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(units=1, activation="sigmoid"))
# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Print the model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 100)           12454000  
                                                                 
 bidirectional (Bidirectiona  (None, 60, 128)          84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 12,637,425
Trainable params: 183,425
Non-trainable params: 12,454,000
_________________________________________________________________
None


In [ ]:
history = model.fit(x_train, y_train, batch_size=2048, epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
622/622 [==============================] - 6221s 10s/step - loss: 0.6549 - accuracy: 0.6055 - val_loss: 0.6375 - val_accuracy: 0.6231
Epoch 2/10
622/622 [==============================] - 5960s 10s/step - loss: 0.6342 - accuracy: 0.6289 - val_loss: 0.6124 - val_accuracy: 0.6537
Epoch 3/10
622/622 [==============================] - 6629s 11s/step - loss: 0.5939 - accuracy: 0.6703 - val_loss: 0.5725 - val_accuracy: 0.6921
Epoch 4/10
622/622 [==============================] - 7258s 12s/step - loss: 0.5670 - accuracy: 0.6960 - val_loss: 0.5636 - val_accuracy: 0.6986
Epoch 5/10
622/622 [==============================] - 7838s 13s/step - loss: 0.5521 - accuracy: 0.7093 - val_loss: 0.5550 - val_accuracy: 0.7072
Epoch 6/10
622/622 [==============================] - 8198s 13s/step - loss: 0.5426 - accuracy: 0.7175 - val_loss: 0.5483 - val_accuracy: 0.7123
Epoch 7/10
622/622 [==============================] - 8622s 14s/step - loss: 0.5373 - accuracy: 0.7223 - val_loss: 0.5331 - val_ac

In [ ]:
model.save("BiLSTM_model.h5")

# Fine Tune Model

In [ ]:
data_EV = pd.read_csv('./Tweets_EV_FineTune.csv',encoding = 'UTF-8')

In [ ]:
data_EV

,tweet_text,sentiment
0,Lithium-ion battery explosions are now the thi...,0
1,"When an Electric Vehicle catches fire, it can’...",0
2,The current Mini electric vehicle is delightfu...,1
3,Vietnamese EV maker VinFast remains optimistic...,1
4,Electric Eco car on fire problem burn damage h...,0
...,...,...
194,i dislike fully electric vehicles only because...,0
195,And the truth is the electrical grid cannot su...,0
196,"IMO, Hydrogen is the way to go over EV for env...",0
197,I wonder how many EV drivers dislike single us...,0


In [ ]:
data_EV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  199 non-null    object
 1   sentiment   199 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [ ]:
data_EV['tweet_text'] = data_EV['tweet_text'].apply(lambda x:preprocessing_tweets((x)))

In [ ]:
data_EV['tweet_text']

0      lithiumion battery explosion third leading fir...
1      electric vehicle catch fire put water foam cau...
2      current mini electric vehicle delightful recen...
3      vietnamese ev maker vinfast remain optimistic ...
4      electric eco car fire problem burn damage hybr...
                             ...                        
194    dislike fully electric vehicle almost get hit ...
195    truth electrical grid cannot support massive i...
196    going hydrogen way go ev environmental reason ...
197    wonder many ev driver dislike single use plast...
198    ev need option manual preconditioning button d...
Name: tweet_text, Length: 199, dtype: object

In [ ]:
def test_output(tweet_text, max_tweet_len):
    X=[]
    for tweet in tqdm(tweet_text):
        X1 = tweet.split(' ')
        with open('Pruned_gloveModel.pkl', 'rb') as f:
            # Create an Unpickler object and call its load() method to unpickle the desired value
            unpickler = pickle.Unpickler(f)
            obj = unpickler.load()
            idx = [obj[word]+1  if word in obj.keys() else obj['unk']+1 for word in X1]
            X.append(idx)
    X = pad_sequences(X, maxlen=max_tweet_len, padding='post', truncating='post', value=0)
    return X

In [ ]:
max_tweet_len = 60
X = test_output(data_EV['tweet_text'], max_tweet_len)

100%|████████████████████████████████████████████████████████████████████████████████| 199/199 [00:07<00:00, 25.49it/s]


In [ ]:
X #TO BE REMOVED

array([[29394,  3365, 11930, ...,     0,     0,     0],
       [ 6581, 10152,  1327, ...,     0,     0,     0],
       [ 3755,  1960,  6581, ...,     0,     0,     0],
       ...,
       [   91, 36942,   122, ...,     0,     0,     0],
       [  813,   343,  8186, ...,     0,     0,     0],
       [ 8186,    64,  4550, ...,     0,     0,     0]])

In [ ]:
Y = getOutput(data_EV['sentiment'])

In [ ]:
# Load pre-trained model
pretrained_model = load_model('BiLSTM_model.h5')
for layer in pretrained_model.layers:
    layer.trainable = False

# Replace the last layer
num_classes = 1
last_layer = pretrained_model.get_layer('bidirectional_1')
last_output = last_layer.output
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='leaky_relu')(x)
x = layers.Dense(128, activation='leaky_relu')(x)
x = layers.Dense(num_classes, activation='sigmoid')(x)
model = Model(pretrained_model.input, x)
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_input (InputLayer  [(None, 60)]             0         
 )                                                               
                                                                 
 embedding (Embedding)       (None, 60, 100)           12454000  
                                                                 
 bidirectional (Bidirectiona  (None, 60, 128)          84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 128)               0         
                                                           

In [ ]:
# Compile the model
num_epochs = 10
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X,Y, epochs=num_epochs)

Epoch 1/10
7/7 [==============================] - 9s 113ms/step - loss: 0.6801 - accuracy: 0.5578
Epoch 2/10
7/7 [==============================] - 1s 107ms/step - loss: 0.6516 - accuracy: 0.6080
Epoch 3/10
7/7 [==============================] - 1s 106ms/step - loss: 0.6318 - accuracy: 0.6633
Epoch 4/10
7/7 [==============================] - 1s 107ms/step - loss: 0.6224 - accuracy: 0.6533
Epoch 5/10
7/7 [==============================] - 1s 102ms/step - loss: 0.6211 - accuracy: 0.6533
Epoch 6/10
7/7 [==============================] - 1s 100ms/step - loss: 0.6195 - accuracy: 0.6533
Epoch 7/10
7/7 [==============================] - 1s 102ms/step - loss: 0.6210 - accuracy: 0.6734
Epoch 8/10
7/7 [==============================] - 1s 102ms/step - loss: 0.7170 - accuracy: 0.5678
Epoch 9/10
7/7 [==============================] - 1s 99ms/step - loss: 0.6641 - accuracy: 0.5879
Epoch 10/10
7/7 [==============================] - 1s 96ms/step - loss: 0.6582 - accuracy: 0.6281


In [ ]:
# Save the fine-tuned model
model.save('fine_tuned_model.h5')